# SCRAPING SET UP

In [53]:
# Import necessary modules
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from datetime import datetime
import pandas as pd
import inspect
import random
import time
import re

def startScraping(url):
    # FUNCTIONS

    # Sleep
    def fSleep(x, y):
        # x must be larger than y
        time.sleep(random.uniform(x, y))

    # VARIABLES

    # Buttons
    button_filter = '//*[@id="filter-search-group"]/button[1]'
    button_all_sources = '/html/body/div[7]/div/div[2]/div[1]/div[1]/div[2]/div[2]/div[2]/button[1]'
    button_save_filters = '/html/body/div[7]/div/div[1]/div[2]/div[2]/button[3]'

    # Lists
    list_parent = '//*[@id="listcontainer"]/div[4]'

    # SCRAPING

    # Open the webpage

    # Instantiate an Options object
    option = webdriver.ChromeOptions()
    #Remove navigator.webdriver flag
    option.add_argument('--disable-blink-features=AutomationControlled')
    # Change the resolution of the browser
    option.add_argument("window-size=1920,1080")
    # Adjusting the user agent
    option.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36")
    #Open Browser
    driver = webdriver.Chrome(executable_path='chromedriver.exe', options=option)
    # Open the specfied URL
    driver.get(url)
    # Sleep to avoid errors
    time.sleep(random.uniform(10.0, 15.0))

    # Set up the filters

    # Open the filter menu
    driver.find_element_by_xpath(button_filter).click()
    # Sleep to avoid errors
    fSleep(3, 5)
    # Set the filter to all sources
    driver.find_element_by_xpath(button_all_sources).click()
    # Sleep to avoid errors
    fSleep(3, 5)
    # Save the filter settings
    driver.find_element_by_xpath(button_save_filters).click()
    # Sleep to avoid errors
    fSleep(3, 5)
    # Print noting success
    print('Filters set to all sources')
    # Return the driver to work with
    return driver, list_parent


# ITEMS

## Scraping

## Data Cleaning
### For cleaning the list that I copy-paste from 5etools

### Magic Items

In [7]:
# Import Pandas
import pandas as pd
# Read the CSV to a DataFrame
df = pd.read_csv('RAW DATA/Item Lists by Rarity - Magical Items.csv', header = None)
# Change the column name
df.columns = ['stuff']
# Create a new DataFrame for the final database
dbMagicItems = pd.DataFrame(columns= ['NAME', 'TYPE', 'WEIGHT', 'ATTUNEMENT', 'RARITY', 'SOURCE', 'LINK'])
# Set a margin to iterate through the index
margin = 0
# Loop through Database's rows and write to them from the CSV
while margin < len(df):
    vName = df['stuff'][0 + margin]
    vType = df['stuff'][1 + margin]
    vWeight = df['stuff'][2 + margin]    
    vAttunement = df['stuff'][3 + margin]
    if vAttunement != '×':
        vAttunement = 'N'
        vRarity = df['stuff'][3 + margin]
        vSource = df['stuff'][4 + margin]
        margin += 5
    else:
        vAttunement = 'Y'
        vRarity = df['stuff'][4 + margin]
        vSource = df['stuff'][5 + margin]
        margin += 6
    # Format the name for the link    
    vNameLink = vName.lower().strip().replace(" ", "%20")
    vNameLink = vNameLink.replace(",", "%2c")
    vNameLink = vNameLink.replace("+", "%2b")
    vNameLink = vNameLink + "_" + vSource.lower()
    # Build the link
    vLink = f'https://5e.tools/items.html#{vNameLink}'
    dbMagicItems = dbMagicItems.append({'NAME': vName, 'TYPE': vType, 'WEIGHT': vWeight, 'ATTUNEMENT': vAttunement, 'RARITY': vRarity, 'SOURCE': vSource, 'LINK': vLink}, ignore_index = True)
# Export to CSV
dbMagicItems.to_csv('DATA OUTPUT/magic_items.csv')

### Mundane (None) Items

In [4]:
# Read the CSV into a DataFrame
dfMundane = pd.read_csv('RAW DATA/Item Lists by Rarity - Mundane Items.csv', header = None)
# Set the colum name
dfMundane.columns = ['stuff']
# Create a new DataFrame for the final database
dbMundane = pd.DataFrame(columns= ['NAME', 'TYPE', 'COST', 'WEIGHT', 'SOURCE', 'LINK'])
# Set a margin to iterate through the index
margin = 0
# Add rows to the DB while the margin is less than the DF's length
while margin < len(dfMundane):
    vName = dfMundane['stuff'][0 + margin]
    vType = dfMundane['stuff'][1 + margin]
    vCost = dfMundane['stuff'][2 + margin]
    vWeight = dfMundane['stuff'][3 + margin]
    vSource = dfMundane['stuff'][4 + margin]
    # Format the name for the link    
    vNameLink = vName.lower().strip().replace(" ", "%20")
    vNameLink = vNameLink.replace(",", "%2c")
    vNameLink = vNameLink.replace("+", "%2b")
    vNameLink = vNameLink + "_" + vSource.lower()
    # Build the link
    vLink = f'https://5e.tools/items.html#{vNameLink}'
    # Append a row to the Database
    dbMundane = dbMundane.append({'NAME': vName, 'TYPE': vType, 'COST': vCost, 'WEIGHT': vWeight, 'SOURCE': vSource, 'LINK': vLink}, ignore_index = True)
    # Increment the margin
    margin += 5
# Export to CSV
dbMundane.to_csv('DATA OUTPUT/mundane_items.csv')

# BESTIARY

## Scraping

## Data Cleaning

In [2]:
# TODO: Need to handle parentheticals, the subtype
    # E.G. Aberration (shapechanger)
        # Move the parenthteical to another column named SUBTYPE

# Import necessary modules
import pandas as pd
import re
# Read the CSV to a DataFrame
df = pd.read_csv('RAW DATA/Bestiary - Sheet1.csv', header = None)
# Change the column name
df.columns = ['stuff']
# Create a new DataFrame for the final database
dbBestiary = pd.DataFrame(columns= ['NAME', 'TYPE', 'SUBTYPE', 'CR', 'SOURCE', 'LINK'])
# Set a margin to iterate through the index
margin = 0
# Loop through Database's rows and write to them from the CSV
while margin < len(df):
    # Get the name
    vName = df['stuff'][0 + margin]
    # Get the Type
    vType = df['stuff'][1 + margin]
    # Parse the Type into Type and Subtype
    if '(' in vType:
        x = re.search('\s\(.*\)', vType)
        y = x.group(0).strip()
        vSubtype = re.sub('\(*\)*', '', y)
        vType = re.sub('\s\(.*\)', '', vType)
    else:
        vType = vType
        vSubtype = '' 
    # Get the CR
    vCR = df['stuff'][2 + margin]
    # Get the source
    vSource = df['stuff'][3 + margin]
    # Format the name for the link    
    vNameLink = vName.lower().strip().replace(" ", "%20")
    vNameLink = vNameLink.replace(",", "%2c")
    vNameLink = vNameLink.replace("+", "%2b")
    vNameLink = vNameLink + "_" + vSource.lower()
    # Build the link
    vLink = f'https://5e.tools/bestiary.html#{vNameLink}'
    dbBestiary = dbBestiary.append({'NAME': vName, 'TYPE': vType, 'SUBTYPE': vSubtype, 'CR': vCR, 'SOURCE': vSource, 'LINK': vLink}, ignore_index = True)
    # Increment the offset
    margin += 4
# Export to CSV
dbBestiary.to_csv('DATA OUTPUT/bestiary.csv', index = False)

# SPELLS

## Scraping

In [94]:
# Set up a DataFrame
dfSpell = pd.DataFrame(columns = ['NAME', 'SOURCE', 'PAGE_NUMBER', 'LEVEL', 'SCHOOL', 'CASTING_TIME', 'RANGE', 'COMPONENTS', 'COMPONENTS_MATERIALS', 'COMPONENETS_COST', 'COMPONENTS_CURRENCY', 'DURATION'])

In [100]:
# Open 5etools.com and set up the filters
driver, list_parent = startScraping("https://5e.tools/spells.html#abi-dalzim's%20horrid%20wilting_xge")

# VARIABLES

# Count the number of rows
myList = driver.find_element_by_xpath(list_parent)
vRowCount = len(myList.find_elements_by_xpath("./*"))
# Name
xName = '//*[@id="pagecontent"]/tr[2]'
# Level & School
xLevelSchool = '//*[@id="pagecontent"]/tr[3]'
# Casting Time
xCastingTime = '//*[@id="pagecontent"]/tr[4]'
# Range
xRange = '//*[@id="pagecontent"]/tr[5]'
# Components
xComponents = '//*[@id="pagecontent"]/tr[6]'
# Duration
xDuration = '//*[@id="pagecontent"]/tr[7]'

# Loop through all the rows and scrape their data
for row in range(1, vRowCount + 1):
    
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # SCRAPING
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~

    # Set up the row based on its index
    xRow = f'//*[@id="listcontainer"]/div[4]/div[{row}]'
    # Click into the row
    driver.find_element_by_xpath(xRow).click()
    # Sleep
    fSleep(0.5, 1)

    # ROW 2

    tName = driver.find_element_by_xpath(xName).text
    workingName = tName.split("\n")
    # Name
    finalName = workingName[0]
    # Source
    finalSource = workingName[1]
    # Page Number
    finalPageNumber = workingName[2]

    # ROW 3

    tLevelSchool = driver.find_element_by_xpath(xLevelSchool).text
    workingLevelSchool = tLevelSchool.split(" ")
    # Level 
    finalLevel = workingLevelSchool[0]
    # School
    finalSchool = workingLevelSchool[1]

    # ROW 4

    tCastingTime = driver.find_element_by_xpath(xCastingTime).text
    workingCastingTime = tCastingTime.split(": ")
    # Casting Time
    finalCastingTime = workingCastingTime[1]

    # ROW 5

    tRange = driver.find_element_by_xpath(xRange).text
    workingRange = tRange.split(": ")
    # Range
    finalRange = workingRange[1]

    # ROW 6

    tComponents = driver.find_element_by_xpath(xComponents).text
    tComponents = tComponents.replace(")", "")
    workingComponents = tComponents.split(": ")
    workingComponents = workingComponents[1].split(" (")
    # Components Required
    finalComponentsRequired = workingComponents[0]
    # Material Components
    try:
        # TODO Pulls out Materials if there is a cost, but not if there is no cost
        finalComponentsMaterial = workingComponents[1]
    except IndexError:
        finalComponentsMaterial = None
    # Currency and Cost
    try:
        if re.search("\d+", finalComponentsMaterial):
            finalCost = re.search("\d+", finalComponentsMaterial)[0]
            # TODO handle more than one cost
            finalCurrency = re.search("(([cesgp]p)($|\s|\W))", finalComponentsMaterial)[0]
        else:
            finalCost = None
            finalCurrency = None
    except:
        finalCost = None
        finalCurrency = None

    # ROW 7

    tDuration = driver.find_element_by_xpath(xDuration).text
    workingDuration = tDuration.split(": ")
    # Duration
    finalDuration = workingDuration[1]

    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # APPEND TO DATAFRAME
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~

    dfSpells = dfSpells.append({'NAME':finalName, 'SOURCE':finalSource, 'PAGE_NUMBER':finalPageNumber, 'LEVEL':finalLevel, 'SCHOOL':finalSchool, 'CASTING_TIME':finalCastingTime, 'RANGE':finalRange, 'COMPONENTS':finalComponentsRequired, 'COMPONENTS_MATERIALS':finalComponentsMaterial, 'COMPONENETS_COST':finalCost, 'COMPONENTS_CURRENCY':finalCurrency, 'DURATION':finalDuration}, ignore_index = True)

Filters set to all sources


KeyboardInterrupt: 

In [102]:
dfSpells.to_csv('SPELLS.csv')

## Data Cleaning

In [6]:
# Import Pandas
import pandas as pd
# Read the CSV to a DataFrame
dfSpells = pd.read_csv('RAW DATA/Spells - Sheet1.csv', header = None)
# Change the column name
dfSpells.columns = ['stuff']
# Create a new DataFrame for the final database
dbSpells = pd.DataFrame(columns= ['NAME', 'LEVEL', 'RITUAL', 'TECHNOLOGY', 'CASTING_TIME', 'SCHOOL', 'CONCENTRATION', 'RANGE', 'SOURCE', 'LINK'])
# Set a margin to iterate through the index
margin = 0
# Loop through Database's rows and write to them from the CSV
while margin < len(dfSpells):
    vName = dfSpells['stuff'][0 + margin]
    vLevel = dfSpells['stuff'][1 + margin]
    # Parse the level
    if '(rit.)' in vLevel:
        x = re.search('\s\(.*\)', vLevel)
        y = x.group(0).strip()
        vRitual = 'Y'
        vLevel = re.sub('\s\(.*\)', '', vLevel)
        vTech = 'N'
    elif '(tec.)' in vLevel:
        x = re.search('\s\(.*\)', vLevel)
        y = x.group(0).strip()
        vTech = 'Y'
        vLevel = re.sub('\s\(.*\)', '', vLevel)
        vRitual = 'N'
    else:
        vLevel = vLevel
        vRitual = 'N'
        vTech = 'N'
    vCastingTime = dfSpells['stuff'][2 + margin]    
    vSchool = dfSpells['stuff'][3 + margin]
    vConcentration = dfSpells['stuff'][4 + margin]
    if vConcentration != '×':
        vConcentration = 'N'
        vRange = dfSpells['stuff'][4 + margin]
        vSource = dfSpells['stuff'][5 + margin]
        margin += 6
    else:
        vConcentration = 'Y'
        vRange = dfSpells['stuff'][5 + margin]
        vSource = dfSpells['stuff'][6 + margin]
        margin += 7
    # Format the name for the link    
    vNameLink = vName.lower().strip().replace(" ", "%20")
    vNameLink = vNameLink.replace(",", "%2c")
    vNameLink = vNameLink.replace("+", "%2b")
    vNameLink = vNameLink + "_" + vSource.lower()
    # Build the link
    vLink = f'https://5e.tools/spells.html#{vNameLink}'
    dbSpells = dbSpells.append({'NAME': vName, 'LEVEL': vLevel, 'RITUAL': vRitual, 'TECHNOLOGY': vTech,'CASTING_TIME': vCastingTime, 'SCHOOL': vSchool, 'CONCENTRATION': vConcentration, 'RANGE': vRange, 'SOURCE': vSource, 'LINK': vLink}, ignore_index = True)
# Export to CSV
dbSpells.to_csv('DATA OUTPUT/spells.csv', index = False)

In [7]:
dbSpells

,NAME,LEVEL,RITUAL,TECHNOLOGY,CASTING_TIME,SCHOOL,CONCENTRATION,RANGE,SOURCE,LINK
0,Abi-Dalzim's Horrid Wilting,8th,N,N,Action,Necro.,N,150 feet,XGE,https://5e.tools/spells.html#abi-dalzim's%20ho...
1,Absorb Elements,1st,N,N,Reaction,Abj.,N,Self,XGE,https://5e.tools/spells.html#absorb%20elements...
2,Acid Splash,Cantrip,N,N,Action,Conj.,N,60 feet,PHB,https://5e.tools/spells.html#acid%20splash_phb
3,Acid Stream,1st,N,N,Action,Evoc.,Y,Self (30-foot line),UA20SMT,https://5e.tools/spells.html#acid%20stream_ua2...
4,Aganazzar's Scorcher,2nd,N,N,Action,Evoc.,N,30 feet,XGE,https://5e.tools/spells.html#aganazzar's%20sco...
...,...,...,...,...,...,...,...,...,...,...
568,Wrathful Smite,1st,N,N,Bonus acn.,Evoc.,Y,Self,PHB,https://5e.tools/spells.html#wrathful%20smite_phb
569,Wristpocket,2nd,Y,N,Action,Conj.,Y,Self,EGW,https://5e.tools/spells.html#wristpocket_egw
570,Zephyr Strike,1st,N,N,Bonus acn.,Trans.,Y,Self,XGE,https://5e.tools/spells.html#zephyr%20strike_xge
571,Zephyr Strike (UA),1st,N,N,Bonus acn.,Trans.,Y,Self,UASS,https://5e.tools/spells.html#zephyr%20strike%2...
